# Mosaic AI Agent Framework: Author and deploy a multi-agent system with Genie

This notebook demonstrates how to build a multi-agent system using Mosaic AI Agent Framework and [LangGraph](https://blog.langchain.dev/langgraph-multi-agent-workflows/), where [Genie](https://www.databricks.com/product/ai-bi/genie) is one of the agents.
In this notebook, you:
1. Author a multi-agent system using LangGraph.
1. Wrap the LangGraph agent with MLflow `ChatAgent` to ensure compatibility with Databricks features.
1. Manually test the multi-agent system's output.
1. Log and deploy the multi-agent system.

This example is based on [LangGraph documentation - Multi-agent supervisor example](https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/multi_agent/agent_supervisor.md)

## Why use a Genie agent?

Multi-agent systems consist of multiple AI agents working together, each with specialized capabilities. As one of those agents, Genie allows users to interact with their structured data using natural language.

Unlike SQL functions which can only run pre-defined queries, Genie has the flexibility to create novel queries to answer user questions.

## Prerequisites

- Address all `TODO`s in this notebook.
- Create a Genie Space, see Databricks documentation ([AWS](https://docs.databricks.com/aws/genie/set-up) | [Azure](https://learn.microsoft.com/azure/databricks/genie/set-up)).

In [0]:
%pip install -U -qqq langgraph-supervisor==0.0.30 mlflow[databricks] databricks-langchain databricks-agents uv 
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



## Define the multi-agent system

Create a multi-agent system in LangGraph using a supervisor agent node directing the following agent nodes:
- **GenieAgent**: The Genie agent that queries and reasons over structured data.
- **Tool-calling agent**: An agent that calls Unity Catalog function tools.

In this example, the tool-calling agent uses the built-in Unity Catalog function `system.ai.python_exec` to execute Python code.
For examples of other tools you can add to your agents, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool)).


#### Wrap the LangGraph agent using the `ChatAgent` interface

Databricks recommends using `ChatAgent` to ensure compatibility with Databricks AI features and to simplify authoring multi-turn conversational agents using an open source standard. 

The `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent.

See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).

#### Write agent code to file

Define the agent code in a single cell below. This lets you write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.


In [0]:
%%writefile agent.py
import json
from typing import Generator, Literal
from uuid import uuid4

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import Runnable
from langchain.agents import create_agent
from langgraph.graph.state import CompiledStateGraph
from langgraph_supervisor import create_supervisor
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from pydantic import BaseModel
from databricks_ai_bridge import ModelServingUserCredentials
from databricks.sdk import WorkspaceClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

########################################
# Create your LangGraph Supervisor Agent
########################################

GENIE = "genie"


class ServedSubAgent(BaseModel):
    endpoint_name: str
    name: str
    task: Literal["agent/v1/responses", "agent/v1/chat", "agent/v2/chat"]
    description: str


class Genie(BaseModel):
    space_id: str
    name: str
    task: str = GENIE
    description: str


class InCodeSubAgent(BaseModel):
    tools: list[str]
    name: str
    description: str


TOOLS = []


def stringify_content(state):
    msgs = state["messages"]
    if isinstance(msgs[-1].content, list):
        msgs[-1].content = json.dumps(msgs[-1].content, indent=4)
    return {"messages": msgs}


def create_langgraph_supervisor(
    llm: Runnable,
    externally_served_agents: list[ServedSubAgent] = [],
    in_code_agents: list[InCodeSubAgent] = [],
):
    agents = []
    agent_descriptions = ""

    # Process inline code agents
    for agent in in_code_agents:
        agent_descriptions += f"- {agent.name}: {agent.description}\n"
        uc_toolkit = UCFunctionToolkit(function_names=agent.tools)
        TOOLS.extend(uc_toolkit.tools)
        agents.append(create_agent(llm, tools=uc_toolkit.tools, name=agent.name))

    # Process served endpoints and Genie Spaces
    for agent in externally_served_agents:
        agent_descriptions += f"- {agent.name}: {agent.description}\n"
        if isinstance(agent, Genie):
            # to better control the messages sent to the genie agent, you can use the `message_processor` param: https://api-docs.databricks.com/python/databricks-ai-bridge/latest/databricks_langchain.html#databricks_langchain.GenieAgent
            genie_agent = GenieAgent(
                genie_space_id=agent.space_id,
                genie_agent_name=agent.name,
                description=agent.description,
                client = WorkspaceClient(credentials_strategy=ModelServingUserCredentials())
            )
            genie_agent.name = agent.name
            agents.append(genie_agent)
        else:
            model = ChatDatabricks(
                endpoint=agent.endpoint_name, use_responses_api="responses" in agent.task
            )
            # Disable streaming for subagents for ease of parsing
            model._stream = lambda x: model._stream(**x, stream=False)
            agents.append(
                create_agent(
                    model,
                    tools=[],
                    name=agent.name,
                    post_model_hook=stringify_content,
                )
            )

    # TODO: The supervisor prompt includes agent names/descriptions as well as general
    # instructions. You can modify this to improve quality or provide custom instructions.
    prompt = f"""
    You are a supervisor in a multi-agent system.

    1. Understand the user's last request
    2. Read through the entire chat history.
    3. If the answer to the user's last request is present in chat history, answer using information in the history.
    4. If the answer is not in the history, from the below list of agents, determine which agent is best suited to answer the question.
    5. Provide a summarized response to the user's last query, even if it's been answered before.

    {agent_descriptions}"""

    return create_supervisor(
        agents=agents,
        model=llm,
        prompt=prompt,
        add_handoff_messages=False,
        output_mode="full_history",
    ).compile()



#######################################################
# Configure the Foundation Model and Serving Sub-Agents
#######################################################

# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-claude-sonnet-4-5"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# TODO: Add the necessary information about each of your subagents. Subagents could be agents deployed to Model Serving endpoints or Genie Space subagents.
# Your agent descriptions are crucial for improving quality. Include as much detail as possible.
EXTERNALLY_SERVED_AGENTS = [
    Genie(
        space_id="01f0d963adac19f9aab30a22ae88978f",
        name="Bakehouse Genie",
        description="This agent can answer questions on the bakehouse data",
    ),
    # ServedSubAgent(
    #     endpoint_name="cities-agent",
    #     name="city-agent", # choose a semantically relevant name for your agent
    #     task="agent/v1/responses",
    #     description="This agent can answer questions about the best cities to visit in the world.",
    # ),
]

############################################################
# Create additional agents in code
############################################################

# TODO: Fill the following with UC function-calling agents. The tools parameter is a list of UC function names that you want your agent to call.
IN_CODE_AGENTS = [
    InCodeSubAgent(
        tools=["system.ai.*"],
        name="code execution agent",
        description="The code execution agent specializes in solving programming challenges, generating code snippets, debugging issues, and explaining complex coding concepts.",
    )
]

##########################################
# Wrap LangGraph Supervisor as a ResponsesAgent
##########################################


class LangGraphResponsesAgent(ResponsesAgent):
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(
        self,
        request: ResponsesAgentRequest,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        #################################################
        # Create supervisor and set up MLflow for tracing
        #################################################
        agent = create_langgraph_supervisor(llm, EXTERNALLY_SERVED_AGENTS, IN_CODE_AGENTS)
        cc_msgs = to_chat_completions_input([i.model_dump() for i in request.input])
        first_message = True
        seen_ids = set()

        # can adjust `recursion_limit` to limit looping: https://docs.langchain.com/oss/python/langgraph/GRAPH_RECURSION_LIMIT#troubleshooting
        for _, events in agent.stream({"messages": cc_msgs}, stream_mode=["updates"]):
            new_msgs = [
                msg
                for v in events.values()
                for msg in v.get("messages", [])
                if msg.id not in seen_ids
            ]
            if first_message:
                seen_ids.update(msg.id for msg in new_msgs[: len(cc_msgs)])
                new_msgs = new_msgs[len(cc_msgs) :]
                first_message = False
            else:
                seen_ids.update(msg.id for msg in new_msgs)
                node_name = tuple(events.keys())[0]  # assumes one name per node
                yield ResponsesAgentStreamEvent(
                    type="response.output_item.done",
                    item=self.create_text_output_item(
                        text=f"<name>{node_name}</name>", id=str(uuid4())
                    ),
                )
            if len(new_msgs) > 0:
                yield from output_to_responses_items_stream(new_msgs)

mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent()
mlflow.models.set_model(AGENT)

Overwriting agent.py


## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

**TODO**: Replace this placeholder `input_example` with a domain-specific prompt for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import AGENT

AGENT.predict({
    "input": [
        {"role": "user", "content": "Which are the most popular cookies by franchise?"}
    ]
})

ResponsesAgentResponse(tool_choice=None, truncation=None, id=None, created_at=None, error=None, incomplete_details=None, instructions=None, metadata=None, model=None, object='response', output=[OutputItem(type='message', id='0c765033-16aa-4391-878c-f7ce8a30d9e7', content=[{'text': '<name>Bakehouse Genie</name>', 'type': 'output_text'}], role='assistant'), OutputItem(type='message', id='0041849e-215a-4ae2-a731-2888fc5e2189', content=[{'text': '|    |   franchiseID | product                |   total_quantity |\n|---:|--------------:|:-----------------------|-----------------:|\n|  0 |       3000000 | Outback Oatmeal        |              190 |\n|  1 |       3000001 | Pearly Pies            |               74 |\n|  2 |       3000002 | Austin Almond Biscotti |              186 |\n|  3 |       3000003 | Pearly Pies            |               69 |\n|  4 |       3000004 | Pearly Pies            |              122 |\n|  5 |       3000005 | Outback Oatmeal        |               97 |\n|  6 |   

Trace(trace_id=tr-5165352c680ace7e67affd253b16e92b)

In [0]:
input_example = {
    "input": [
        {"role": "user", "content": "Which cookies are the most popular?"}
    ]
}
AGENT.predict(input_example)

ResponsesAgentResponse(tool_choice=None, truncation=None, id=None, created_at=None, error=None, incomplete_details=None, instructions=None, metadata=None, model=None, object='response', output=[OutputItem(type='message', id='9c69f8d1-61fd-4023-a0a3-998d28309775', content=[{'text': '<name>Bakehouse Genie</name>', 'type': 'output_text'}], role='assistant'), OutputItem(type='message', id='ab1836c1-d0ca-4eba-a53f-4e125d313a8f', content=[{'text': '|    | product                |   total_quantity_sold |\n|---:|:-----------------------|----------------------:|\n|  0 | Golden Gate Ginger     |                  3865 |\n|  1 | Outback Oatmeal        |                  3733 |\n|  2 | Austin Almond Biscotti |                  3716 |\n|  3 | Tokyo Tidbits          |                  3662 |\n|  4 | Pearly Pies            |                  3595 |\n|  5 | Orchard Oasis          |                  3586 |', 'type': 'output_text'}], role='assistant'), OutputItem(type='message', id='fae13c86-8bfd-46fa-9b

Trace(trace_id=tr-14e6f7b53590ec55413b2c6ae6751dec)

In [0]:
for event in AGENT.predict_stream(input_example):
  print(event, "-----------\n")

type='response.output_item.done' custom_outputs=None item={'id': '5ff37d45-8e8b-43ef-bed2-ae8fcab6a19b', 'content': [{'text': '<name>Bakehouse Genie</name>', 'type': 'output_text'}], 'role': 'assistant', 'type': 'message'} -----------

type='response.output_item.done' custom_outputs=None item={'id': '6f353239-793c-4c46-9af2-7df24a31ecc2', 'content': [{'text': '|    | product                |   total_quantity |\n|---:|:-----------------------|-----------------:|\n|  0 | Golden Gate Ginger     |             3865 |\n|  1 | Outback Oatmeal        |             3733 |\n|  2 | Austin Almond Biscotti |             3716 |\n|  3 | Tokyo Tidbits          |             3662 |\n|  4 | Pearly Pies            |             3595 |\n|  5 | Orchard Oasis          |             3586 |', 'type': 'output_text'}], 'role': 'assistant', 'type': 'message'} -----------

type='response.output_item.done' custom_outputs=None item={'id': '7597c63f-70a5-47c0-810e-6ec585cc1792', 'content': [{'text': '<name>superviso

Trace(trace_id=tr-a083771b658178ddfe68ce0537f9e0d1)

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`
  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/log-agent#resources) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).

  - **TODO**: If the SQL Warehouse powering your Genie space has secured permissions, include the warehouse ID and table name in your resources to enable passthrough authentication. ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/log-agent#-specify-resources-for-automatic-authentication-passthrough-system-authentication) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#-specify-resources-for-automatic-authentication-passthrough-system-authentication)).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from mlflow.models.auth_policy import AuthPolicy, SystemAuthPolicy, UserAuthPolicy
from databricks_langchain import UnityCatalogTool, VectorSearchRetrieverTool
from mlflow.models.resources import (
    DatabricksFunction,
    DatabricksGenieSpace,
    DatabricksServingEndpoint,
#   DatabricksSQLWarehouse,
#   DatabricksTable,
)
from pkg_resources import get_distribution

# TODO: Manually include underlying resources if needed. See the TODO in the markdown above for more information.
resources = [
    DatabricksServingEndpoint(endpoint_name="databricks-claude-sonnet-4-5"),
]
resources.append(DatabricksFunction(function_name="system.ai.python_exec"))
        
systemAuthPolicy = SystemAuthPolicy(resources=resources)
userAuthPolicy = UserAuthPolicy(api_scopes=["serving.serving-endpoints", "dashboards.genie"])

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        auth_policy=AuthPolicy(system_auth_policy=systemAuthPolicy, user_auth_policy=userAuthPolicy),
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
            f"langgraph-supervisor=={get_distribution('langgraph-supervisor').version}",
        ],
    )

🔗 View Logged Model at: https://adb-122999858026867.7.azuredatabricks.net/ml/experiments/1625706306455247/models/m-4682838a3de84f1e8351344649c7fc0d?o=122999858026867
/databricks/python/lib/python3.12/site-packages/databricks/connect/session.py:477: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/12/16 22:19:53 INFO mlflow.pyfunc: Predicting on input example to validate output
2025/12/16 22:19:53 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/12/16 22:19:54 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
/databricks/python/lib/python3

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data=input_example,
    env_manager="uv",
)

2025/12/16 22:20:31 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2025/12/16 22:20:32 INFO mlflow.utils.virtualenv: Environment /tmp/virtualenv_envs/mlflow-ea8489dea70a28cd5dc94ef906e24460a6d08370 already exists
2025/12/16 22:20:33 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-ea8489dea70a28cd5dc94ef906e24460a6d08370/bin/activate && python -c ""']'
2025/12/16 22:20:33 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-ea8489dea70a28cd5dc94ef906e24460a6d08370/bin/activate && python /local_disk0/.ephemeral_nfs/envs/pythonEnv-5a84e11b-5717-45e7-b476-109a778f337f/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///local_disk0/user_tmp_data/spark-5a84e11b-5717-45e7-b476-10/tmptlor54ma/agent --content-type json --input-path /local_disk0/user_tmp_data/spark-5a84e11b-5717-45e7-b476-10/tmpnn7agnlc/input.json']'


{"object": "response", "output": [{"type": "message", "id": "ec711b61-6501-4748-8b7f-c78733a481d8", "content": [{"text": "<name>Bakehouse Genie</name>", "type": "output_text"}], "role": "assistant"}, {"type": "message", "id": "ef245beb-f466-4baf-9c4b-93fe0fe68ae4", "content": [{"text": "|    | product                |   total_quantity_sold |\n|---:|:-----------------------|----------------------:|\n|  0 | Golden Gate Ginger     |                  3865 |\n|  1 | Outback Oatmeal        |                  3733 |\n|  2 | Austin Almond Biscotti |                  3716 |\n|  3 | Tokyo Tidbits          |                  3662 |\n|  4 | Pearly Pies            |                  3595 |", "type": "output_text"}], "role": "assistant"}, {"type": "message", "id": "0ac94093-c214-4d6a-bcf8-aa3dabcb69d2", "content": [{"text": "<name>supervisor</name>", "type": "output_text"}], "role": "assistant"}, {"type": "message", "id": "lc_run--019b2940-80dc-71b3-87c1-c22671448d6c-0", "content": [{"text": "\n\nBa

2025/12/16 22:20:55 INFO mlflow.tracing.export.async_export_queue: Flushing the async trace logging queue before program exit. This may take a while...


## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "ian_ward"
schema = "bakehouse"
model_name = "bakehouse-obo-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

Registered model 'ian_ward.bakehouse.bakehouse-obo-agent' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/13 [00:00<?, ?it/s]

🔗 Created version '1' of model 'ian_ward.bakehouse.bakehouse-obo-agent': https://adb-122999858026867.7.azuredatabricks.net/explore/data/models/ian_ward/bakehouse/bakehouse-obo-agent/version/1?o=122999858026867


## Deploy the agent

In [0]:
from databricks import agents

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "docs"},
)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-5a84e11b-5717-45e7-b476-109a778f337f/lib/python3.12/site-packages/databricks/agents/deployments.py:641: UserWarning: This endpoint is being deployed without a feedback model, which has been deprecated.
For more information, see: https://docs.databricks.com/aws/en/generative-ai/agent-framework/feedback-model
  warnings.warn(



    Deployment of ian_ward.bakehouse.bakehouse-obo-agent version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-122999858026867.7.azuredatabricks.net/ml/endpoints/agents_ian_ward-bakehouse-bakehouse-obo-agent/?o=122999858026867
    Review App: https://adb-122999858026867.7.azuredatabricks.net/ml/review-v2/f571ddbb30cf402484aa9e6e1800edc5/chat?o=122999858026867

You can refer back to the links above from the endpoint detail page at https://adb-122999858026867.7.azuredatabricks.net/ml/endpoints/agents_ian_ward-bakehouse-bakehouse-obo-agent/?o=122999858026867.

To set up monitoring for your deployed agent, see:
https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/production-monitoring


Deployment(model_name='ian_ward.bakehouse.bakehouse-obo-agent', model_version='1', endpoint_name='agents_ian_ward-bakehouse-bakehouse-obo-agent', served_entity_name='ian_ward-bakehouse-bakehouse-obo-agent_1', query_endpoint='https://adb-122999858026867.7.azuredatabricks.net/serving-endpoints/agents_ian_ward-bakehouse-bakehouse-obo-agent/served-models/ian_ward-bakehouse-bakehouse-obo-agent_1/invocations?o=122999858026867', endpoint_url='https://adb-122999858026867.7.azuredatabricks.net/ml/endpoints/agents_ian_ward-bakehouse-bakehouse-obo-agent/?o=122999858026867', review_app_url='https://adb-122999858026867.7.azuredatabricks.net/ml/review-v2/f571ddbb30cf402484aa9e6e1800edc5/chat?o=122999858026867')

In [0]:
UC_MODEL_NAME

'ian_ward.bakehouse.bakehouse-obo-agent'

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).